# 🌌 Master Notebook – Pipeline Spectroscopie DR5

## Objectif du pipeline

- Ce notebook télécharge, prépare et journalise des spectres .fits.gz de LAMOST DR5 pour entraîner un modèle de classification.
- Le DatasetBuilder garantit qu'aucun spectre ne sera jamais réutilisé en s'appuyant sur un log de fichiers déjà traités.
- Ce pipeline permet d’ajouter progressivement des spectres au jeu d’entraînement, en assurant qu’aucun spectre ne soit traité deux fois. Chaque exécution sélectionne un nouveau lot, l’entraîne, puis marque les spectres comme utilisés.


#

## Étape 0 : SETUP & IMPORTS

In [ ]:
from utils import setup_project_env, ensure_dir, load_env_vars
from pipeline.master import MasterPipeline
from astroquery.gaia import Gaia

# Initialisation automatique de l'environnement et des chemins
paths = setup_project_env()

# Chargement des credentials Gaia depuis .env
env_vars = load_env_vars()

try:
    print("Tentative de connexion à l'archive Gaia...")
    Gaia.login(user=env_vars.get("GAIA_USER"), password=env_vars.get("GAIA_PASS"))
    print("Connexion à Gaia réussie.")
except Exception as e:
    print(f"AVERTISSEMENT : Échec de la connexion à Gaia ({e}). Le mode 'bulk' pourrait échouer.")

# Instanciation du pipeline maître
pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("\nSetup terminé. Tu es prêt à lancer ton pipeline.")

#

---

## Lancer une Session d'Entraînement Complète

In [ ]:
pipeline.run_full_pipeline(batch_size=5000, enrich_gaia=True, model_type='XGBoost')

#

---

#

## 1) Téléchargement des spectres
Utilisation du script ``dr5_downloader.py`` encapsulé en fonction.

Cette étape est désormais externalisée dans [01_download_spectra.ipynb](./01_download_spectra.ipynb) pour être exécutée seulement au besoin.

#

## 2) Préparation des données spectrales : Sélection du lot de spectres à traiter et Enrichissement Gaia et Traitement des Features

In [ ]:
# Pour définir la grosseur du lot modifier la variable de batch_size= par la valeur voulu
pipeline.select_batch(batch_size=1000)

# Version AVEC Gaia - utiliser soit cone ou bulk
pipeline.generate_and_enrich_catalog(
    enrich_gaia=True,
    mode='bulk',
    include_risky=False,   # <-- active radius/mass/age -- beta donc en test
    ruwe_max=9.0            # optionnel: garde aussi les entrées à RUWE élevé
)

pipeline.process_data()

#

## 3) Entraînement du modèle
### Machine Learning
Sélection des features et entraînement d'un modèle de classification.

In [ ]:
pipeline.interactive_training_runner()